In [1]:
from IPython.core.display import HTML

In [2]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_9795/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [3]:
llm.invoke('hi')

'Hello! How can I assist you today?'

In [4]:
llm.invoke('سلام. خوبی؟')

'سلام! تازه\u200cای اگر به خیر هستم، روزتون چطور است؟ اگر کاری دارید یا سوالی بخواهید پرسید، خوشحال می\u200cشوم که در معرض شما باشم.'

In [5]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [6]:
llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [7]:
# llm_math.run("what is 13 raised to the .3432 power?")

In [8]:
# i = 0
# example_query = "What is 551368 divided by 82"
# llm_math.invoke(example_query)

In [9]:
from langchain.chains import LLMMathChain
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# Create a custom prompt template to enforce the expected output.
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a expert math solver. Solve the problem below and output only the final answer in the exact format:\n"
        "Answer: <result>\n\n"
        "Do not include any explanation, commentary, or extra text.\n\n"
        "Round the result with 2. For example,if the result is 5.6666666, make the result 5.66.\n\n"
        "Problem: {question}"
    )
)


# Use your local Ollama model.
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434", temperature = 0)

# Instantiate the math chain with the custom prompt.
math_chain = LLMMathChain(llm=llm, prompt=custom_prompt, verbose = True)

# Run the chain with the math problem.
result = math_chain.invoke("What is 551368 divided by 82?")
# result = math_chain.invoke("What is 551121 divided by 82?")
# result = math_chain.invoke("What is (12 plus 18 minus 10) devide by 3?")
print(result)  # Expected output: Answer: 6724


/tmp/ipykernel_9795/3434510964.py:22: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  math_chain = LLMMathChain(llm=llm, prompt=custom_prompt, verbose = True)
/home/mahdi/Math_LangChain/.venv/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 551368 divided by 82?Answer: 6724.39
> Finished chain.
{'question': 'What is 551368 divided by 82?', 'answer': 'Answer: 6724.39'}


#### The correct answer is 6724.0 !!!!

In [10]:
print("LLMMathChain input keys:", llm_math.input_keys)


LLMMathChain input keys: ['question']


In [11]:
import numexpr as ne
computed_result = ne.evaluate("551368 / 82")
print("Computed numeric result:", computed_result)


Computed numeric result: 6724.0


## understand which formula is needed

<h3>Using Router is a common and effective pattern in LangChain for handling multi-step workflows.</h3> 

<p>Using a router chain that outputs a structured dictionary—typically with keys like "destination" (to select the appropriate chain) and "next_inputs" (to pass along additional context or parameters)—helps separate the decision logic from the execution logic.</p>



<h3>Here’s why this approach is beneficial:</h3>
<ul>
    <li><strong>Modularity: </strong> It decouples the routing logic from the individual chain implementations, allowing each chain to focus on its specific task.</li>
    <li><strong>Scalability: </strong> You can easily add or modify destination chains without changing the overall structure.</li>
    <li><strong>Clarity: </strong> The structured output makes it explicit which chain is being selected and what inputs will be passed along, which is useful for debugging and maintenance.</li>
</ul>


<p>This pattern is often recommended in LangChain examples and documentation for multi-step reasoning tasks.</p>

### Input Libraries

In [1]:
import json
import re
from langchain.schema import BaseOutputParser
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
import math

### Define LLM

In [2]:
# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_409684/2906811197.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


### Define Output Parser for Router_Prompt

In [3]:
# Custom Output Parser Definition

def clean_keys(d: dict) -> dict:
    new_d = {}
    for key, value in d.items():
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys to remove any extra quotes
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"

# Instantiate the custom output parser
router_output_parser = CustomRouterOutputParser()

### Routing Logic and Formula Chain

In [4]:

# Router Prompt: Including new formulas

router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which formula chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: one of 'score_calculation', 'interest_calculation', or 'loan_calculation'.\n"
        "   - Use 'score_calculation' for questions about calculating customer score. \n"
        "   - Use 'interest_calculation' for questions about calculating deposit interest. \n"
        "   - Use 'loan_calculation' for questions about calculating remaining money for loans.\n"
        "2. next_inputs: a dictionary that includes the key 'question' mapping to the original question text.\n"
        "For example, if the question is 'How do I calculate the customer score?', return:\n"
        "{{\"destination\": \"score_calculation\", \"next_inputs\": {{\"question\": \"How do I calculate the customer score?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser
)


### Formula Chain --> Core of Calculations

In [26]:

# Create Formula Chains for Each Task/Formula


# Formula 1: Customer Score Calculation
score_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Customer Score Calculation:\n"
        "For the question: {question}, please calculate the customer's score using the following formula:\n\n"
        "    score = (amount_of_income * month_of_income) - ((years_in_bank)**3 / age)\n\n"
        "Where:\n"
        "  - amount_of_income: The total income amount of the customer.\n"
        "  - month_of_income: The number of months corresponding to the income period.\n"
        "  - years_in_bank: The number of years the customer has held an account with our bank.\n"
        "  - age: The age of the customer.\n\n"
        "Provide only the arithmetic expression using the variable names exactly as defined (e.g., amount_of_income, month_of_income, years_in_bank, age) and do not include any numeric literals as defaults.\n"
        "Provide only the arithmetic expression (without additional explanation) in a format that is compatible with Python's eval().\n"
        "For example, if amount_of_income=7000, month_of_income=9, years_in_bank=10, and age=40, output:\n"
        "(amount_of_income * month_of_income) - ((years_in_bank)**3 / age)"

    )
)
#score = math.sqrt(amount_of_income * month_of_income) - ((years_in_bank)**3 / age)
# "math.sqrt(7000 * 9) - ((10)**3 / 40)"

score_calculation_chain = LLMChain(llm=llm, prompt=score_calculation_prompt, verbose=True)

# Formula 2: Deposit Interest Calculation
interest_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Deposit Interest Calculation:\n"
        "For the question: {question}, calculate the deposit interest using the formula:\n"
        "    interest = ((deposit_amount * months) / 12) * 0.25\n"
        "where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.\n"
        "Provide only the arithmetic expression using the variable names exactly as defined (e.g., deposit_amount, months) and do not include any numeric literals as defaults.\n"
        "Provide only the arithmetic expression that calculates the interest.\n"
        "For example, if deposit_amount=2000000 and months=9, output something like: ((deposit_amount * months) / 12) * 0.25"
    )
)
interest_calculation_chain = LLMChain(llm=llm, prompt=interest_calculation_prompt, verbose=True)

# Formula 3: Loan Calculation Chain (unchanged)
loan_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Loan Calculation Expert:\n"
        "For the question: {question}, please calculate the amount of money required to pay off all loans using the following formula:\n\n"
        "    amount_to_pay = total_amount - (number_of_paid_loans * amount_per_loan)\n\n"
        "Where:\n"
        "  - total_amount: The total amount of money owed on all loans.\n"
        "  - number_of_paid_loans: The number of loans that have already been paid off.\n"
        "  - amount_per_loan: The fixed payment amount for each paid loan.\n\n"
        "Provide only the arithmetic expression using the variable names exactly as defined (e.g., total_amount, number_of_paid_loans, amount_per_loan) and do not include any numeric literals as defaults and any additional text.\n"
        "Provide only the arithmetic expression (without any additional explanation) that calculates amount_to_pay.\n"
        "For example, if total_amount=2000000, number_of_paid_loans=9, and amount_per_loan=200, output:\n"
        "    total_amount - (number_of_paid_loans * amount_per_loan)"
    )
)

loan_calculation_chain = LLMChain(llm=llm, prompt=loan_calculation_prompt, verbose=True)


### Create Router Chain According to Router_Prompt and Destination_Chain

In [6]:
# Create the Router Chain with the Three Destination Chains

destination_chains = {
    "score_calculation": score_calculation_chain,
    "interest_calculation": interest_calculation_chain,
    "loan_calculation": loan_calculation_chain,
}

router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt,
    destination_chains=destination_chains
)

### Examples

In [7]:
# Example Usage: Testing Different Questions


# 1. Customer Score Calculation Question (Formula 1)
input_text1 = "How do I calculate the customer score?"
router_result1 = router_chain({"question": input_text1})
print("Router output for customer score:", router_result1)
destination1 = router_result1["destination"]
next_inputs1 = router_result1["next_inputs"]
if destination1 in destination_chains:
    final_result1 = destination_chains[destination1](next_inputs1)
    print(f"Final result from '{destination1}':", final_result1)
else:
    print("Destination not found:", destination1)


/tmp/ipykernel_409684/3651798248.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result1 = router_chain({"question": input_text1})


Router output for customer score: {'question': 'How do I calculate the customer score?', 'destination': 'score_calculation', 'next_inputs': {'question': 'How do I calculate the customer score?'}}


> Entering new LLMChain chain...
Prompt after formatting:
Customer Score Calculation:
For the question: How do I calculate the customer score?, please calculate the customer's score using the following formula:

    score = (amount_of_income * month_of_income) - ((years_in_bank)**3 / age)

Where:
  - amount_of_income: The total income amount of the customer.
  - month_of_income: The number of months corresponding to the income period.
  - years_in_bank: The number of years the customer has held an account with our bank.
  - age: The age of the customer.

Provide only the arithmetic expression using the variable names exactly as defined (e.g., amount_of_income, month_of_income, years_in_bank, age) and do not include any numeric literals as defaults.
Provide only the arithmetic expression (wit

In [8]:
# 2. Deposit Interest Calculation Question (Formula 2)
# input_text2 = "How do I calculate the deposit interest?"
input_text2 = "من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره"
router_result2 = router_chain({"question": input_text2})
print("Router output for deposit interest:", router_result2)
destination2 = router_result2["destination"]
print(f"destination2 {destination2}")
next_inputs2 = router_result2["next_inputs"]
print(f"next_input2 {next_inputs2}")
if destination2 in destination_chains:
    final_result2 = destination_chains[destination2](next_inputs2)
    print(f"destination_chain2 = {destination_chains[destination2]}")
    print(f"Final result from '{destination2}':", final_result2)
else:
    print("Destination not found:", destination2)

/tmp/ipykernel_7293/1000696705.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result2 = router_chain({"question": input_text2})


Router output for deposit interest: {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره', 'destination': 'interest_calculation', 'next_inputs': {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}}
destination2 interest_calculation
next_input2 {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}


> Entering new LLMChain chain...
Prompt after formatting:
Deposit Interest Calculation:
For the question: من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره, calculate the deposit interest using the formula:
    interest = ((deposit_amount * months) / 12) * 0.25
where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.
Explain the calculation step by step.

> Finished chain.
destination_chain2 = verbose=True prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="Deposit Interest Calculatio

In [7]:
# 3. Loan Calculation Question (Loan Calculation)
# input_text3 = "How much money should I pay to pay all my loans to the bank?"
input_text3 = "How was the weather testerday?"
router_result3 = router_chain({"question": input_text3})
print("Router output for loan calculation:", router_result3)
destination3 = router_result3["destination"]
next_inputs3 = router_result3["next_inputs"]
if destination3 in destination_chains:
    final_result3 = destination_chains[destination3](next_inputs3)
    print(f"Final result from '{destination3}':", final_result3)
else:
    print("Destination not found:", destination3)

/tmp/ipykernel_395243/1638354673.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result3 = router_chain({"question": input_text3})


Router output for loan calculation: {'question': 'How was the weather testerday?', 'destination': None, 'next_inputs': {'question': 'How was the weather yesterday?'}}
Destination not found: None


### Try to use python codes instead of mathchain to solve mathematics part

#### Consider to get missing parameters

## Get missing params from user

In [8]:
# Define required parameters for each formula destination.
formula_required_params = {
    "interest_calculation": ["deposit_amount", "months"],
    "loan_calculation": ["total_amount", "number_of_paid_loans", "amount_per_loan"],
    "score_calculation": ["amount_of_income", "month_of_income","years_in_bank", "age"]
}

#### Previous Version of Extract parameters

In [9]:
# def extract_parameters_generic(prompt: str, required_params: list) -> dict:
    
#     extracted = {}
#     numbers = re.findall(r"([\d,\.]+)", prompt)
#     # Remove commas
#     numbers = [num.replace(',', '') for num in numbers]
#     for i, param in enumerate(required_params):
#         if i < len(numbers):
#             try:
#                 extracted[param] = float(numbers[i])
#             except ValueError:
#                 pass
#     return extracted

#### New Version: Using paterns (regex) to connect numbers to parameters. 

In [10]:

# def persian_to_english_digits(s: str) -> str:
#     """Convert Persian digits to English digits."""
#     mapping = str.maketrans('۰۱۲۳۴۵۶۷۸۹', '0123456789')
#     return s.translate(mapping)

# def extract_parameter(prompt: str, patterns: list) -> float:
#     """
#     Try a list of regex patterns and return the first matched number as a float.
#     The patterns should contain one capturing group that extracts the number.
#     """
#     for pattern in patterns:
#         match = re.search(pattern, prompt)
#         if match:
#             number_str = match.group(1)
#             number_str = persian_to_english_digits(number_str)
#             try:
#                 return float(number_str)
#             except ValueError:
#                 continue
#     return None

# # extract_parameters_with_keywords

# def extract_parameters_generic(prompt: str, required_params: list) -> dict:
#     """
#     Extract parameters based on keywords in the prompt.
    
#     For this example, we assume:
#       - 'years_in_bank' is near the word "سال"
#       - 'amount_of_income' is near "درآمد" or "میلیون"
#       - 'month_of_income' is near "ماه"
#       - 'age' is near "سن"
#     """
#     # Define patterns for each parameter.
#     patterns = {
#         'years_in_bank': [
#             r'(\d+(?:[.,]\d+)?)\s*سال'
#         ],
#         'amount_of_income': [
#             r'درآمد\s*(?:من\s*معادل\s*)?(\d+(?:[.,]\d+)?)',
#             r'(\d+(?:[.,]\d+)?)\s*میلیون'
#         ],
#         'month_of_income': [
#             r'(\d+(?:[.,]\d+)?)\s*ماه'
#         ],
#         'age': [
#             r'(\d+(?:[.,]\d+)?)\s*سن'
#         ]
#     }
    
#     extracted = {}
#     for param in required_params:
#         if param in patterns:
#             value = extract_parameter(prompt, patterns[param])
#             if value is not None:
#                 extracted[param] = value
#     return extracted

# # # Example usage:
# # user_input = "من ۳ ساله که توی بانک حساب دارم. درآمد من معادل ۵۵۰۰ میلیون تومانه. امتیاز من در بانک چقدر میشه؟"
# # required_params = ['amount_of_income', 'month_of_income', 'years_in_bank', 'age']

# # provided = extract_parameters_generic(user_input, required_params)
# # print("Extracted parameters:", provided)


#### Using LLM chains for extracting parameters 

In [11]:
# import json
# import re
# from langchain.llms import Ollama
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain

# # Set up your LLM (using Ollama as an example)
# llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

# ----- 1. Interest Calculation Parameter Extraction -----
interest_extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input for interest calculation:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- deposit_amount: The amount of money deposited.\n"
        "- months: The number of months the money is deposited.\n\n"
        "If a parameter is not mentioned, set its value to null.\n"
        "Example output:\n"
        "{{\n"
        '  "deposit_amount": 500000000,\n'
        '  "months": 7\n'
        "}}"
    )
)
interest_extraction_chain = LLMChain(llm=llm, prompt=interest_extraction_prompt, verbose=True)

# ----- 2. Score Calculation Parameter Extraction -----
score_extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input for customer score calculation:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- amount_of_income: The total income amount of the customer.\n"
        "- month_of_income: The number of months corresponding to the income period.\n"
        "- years_in_bank: The number of years the customer has been with the bank.\n"
        "- age: The age of the customer.\n\n"
        "If a parameter is not mentioned, set its value to null.\n"
        "Example output:\n"
        "{{\n"
        '  "amount_of_income": 500,\n'
        '  "month_of_income": 8,\n'
        '  "years_in_bank": 3,\n'
        '  "age": 30\n'
        "}}"
    )
)
score_extraction_chain = LLMChain(llm=llm, prompt=score_extraction_prompt, verbose=True)

# ----- 3. Loan Calculation Parameter Extraction -----
loan_extraction_prompt = PromptTemplate(
    input_variables=["user_input"],
    template=(
        "Given the following user input for loan calculation:\n"
        "\"{user_input}\"\n\n"
        "Extract the following parameters as a JSON object:\n"
        "- total_amount: The total amount of money owed on all loans.\n"
        "- number_of_paid_loans: The number of loans that have been paid off.\n"
        "- amount_per_loan: The fixed payment amount for each paid loan.\n\n"
        "If a parameter is not mentioned, set its value to null.\n"
        "Example output:\n"
        "{{\n"
        '  "total_amount": 2000000,\n'
        '  "number_of_paid_loans": 9,\n'
        '  "amount_per_loan": 200\n'
        "}}"
    )
)
loan_extraction_chain = LLMChain(llm=llm, prompt=loan_extraction_prompt, verbose=True)

# ----- 4. Robust Parsing of LLM Output -----
def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the JSON object from the LLM output.
    This function looks for the first '{' and the last '}' in the text
    and then attempts to parse that substring as JSON.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception as e:
        raise ValueError(f"Error parsing JSON: {e}")


In [11]:
# Interest Extraction
user_input_interest = "من میخوام ببینم سود سپرده بانک اگه 60700000 پول بخوابونم چقدره"
interest_extraction_result = interest_extraction_chain.run({"user_input": user_input_interest})
print("Interest Extraction Result:")
print(interest_extraction_result)
interest_params = parse_extraction_result(interest_extraction_result)
print("Extracted Interest Parameters:", interest_params)


/tmp/ipykernel_221538/2080078535.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  interest_extraction_result = interest_extraction_chain.run({"user_input": user_input_interest})




> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input for interest calculation:
"من میخوام ببینم سود سپرده بانک اگه 60700000 پول بخوابونم چقدره"

Extract the following parameters as a JSON object:
- deposit_amount: The amount of money deposited.
- months: The number of months the money is deposited.

If a parameter is not mentioned, set its value to null.
Example output:
{
  "deposit_amount": 500000000,
  "months": 7
}

> Finished chain.
Interest Extraction Result:
To extract the parameters from the user input, we need to analyze the sentence provided:

"من میخوام ببینم سود سپرده بانک اگه 60700000 پول بخوابونم چقدره"

1. **Deposit Amount**: The sentence mentions "60700000 پول," which translates to "60,700,000 money." This is the deposit amount.

2. **Months**: There is no mention of a specific number of months in the sentence.

Based on this analysis, we can construct the JSON object as follows:

```json
{
  "deposit_amount": 60700000,
  "months": n

In [11]:
# Score Extraction
user_input_score = "من 3 ساله که توی بانک حساب دارم. درآمد من معادل 9230 میلیون تومانه. امتیاز من در بانک چقدر میشه؟"
score_extraction_result = score_extraction_chain.run({"user_input": user_input_score})
print("Score Extraction Result:")
print(score_extraction_result)
score_params = parse_extraction_result(score_extraction_result)
print("Extracted Score Parameters:", score_params)




> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input for customer score calculation:
"من 3 ساله که توی بانک حساب دارم. درآمد من معادل 9230 میلیون تومانه. امتیاز من در بانک چقدر میشه؟"

Extract the following parameters as a JSON object:
- amount_of_income: The total income amount of the customer.
- month_of_income: The number of months corresponding to the income period.
- years_in_bank: The number of years the customer has been with the bank.
- age: The age of the customer.

If a parameter is not mentioned, set its value to null.
Example output:
{
  "amount_of_income": 500,
  "month_of_income": 8,
  "years_in_bank": 3,
  "age": 30
}

> Finished chain.
Score Extraction Result:
To extract the parameters from the given user input for customer score calculation, we analyze the text:

"من 3 ساله که توی بانک حساب دارم. درآمد من معادل 9230 میلیون تومانه."

From this text, we can identify:

- **years_in_bank**: The user mentions "3 ساله" which translates t

In [12]:
# Loan Extraction
user_input_loan = "How much money should I pay to pay all my loans to the bank?"
loan_extraction_result = loan_extraction_chain.run({"user_input": user_input_loan})
print("Loan Extraction Result:")
print(loan_extraction_result)
loan_params = parse_extraction_result(loan_extraction_result)
print("Extracted Loan Parameters:", loan_params)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following user input for loan calculation:
"How much money should I pay to pay all my loans to the bank?"

Extract the following parameters as a JSON object:
- total_amount: The total amount of money owed on all loans.
- number_of_paid_loans: The number of loans that have been paid off.
- amount_per_loan: The fixed payment amount for each paid loan.

If a parameter is not mentioned, set its value to null.
Example output:
{
  "total_amount": 2000000,
  "number_of_paid_loans": 9,
  "amount_per_loan": 200
}

> Finished chain.
Loan Extraction Result:
To extract the parameters from the given user input for loan calculation, we need to identify any specific numbers related to total loan amount, number of paid loans, and payment per loan. Since these details are not provided in the question:

```json
{
  "total_amount": null,
  "number_of_paid_loans": null,
  "amount_per_loan": null
}
```

This JSON object indicates that no

#### Find missing parameters

In [12]:
def get_missing_parameters(required_params: list, provided: dict) -> list:
    return [p for p in required_params if p not in provided]

In [13]:
def extract_arithmetic_expression(text: str) -> str:

    # Look for a LaTeX math block: \[ ... \]
    latex_match = re.search(r"\\\[(.*?)\\\]", text, re.DOTALL)
    if latex_match:
        expr = latex_match.group(1)
        # Remove newline characters and extra spaces.
        expr = expr.replace("\n", " ").strip()
        # Remove any \text{...} wrappers.
        expr = re.sub(r"\\text\{(.*?)\}", r"\1", expr)
        # If there's an equals sign, take only the part after it.
        if "=" in expr:
            expr = expr.split("=", 1)[1].strip()
        # Replace LaTeX \times with Python's *
        expr = expr.replace(r"\times", "*")
        # Remove extra curly braces.
        expr = expr.replace("{", "").replace("}", "")
        # Remove backslashes preceding underscores.
        expr = re.sub(r"\\_", "_", expr)
        return expr

    # Fallback: search for a line that looks like an arithmetic expression.
    for line in text.splitlines():
        line = line.strip()
        # A simple heuristic: line contains numbers, letters, and arithmetic operators.
        if re.match(r"^[\d\w\.\+\-\*/\(\)\s]+$", line):
            return line
    # If nothing seems to match, return the original text.
    return text


### Example

In [18]:
# user_input = "من میخوام ببینم سود سپرده بانک اگه 500000000 پول بخوابونم چقدره"
# user_input = "How do I calculate my customer score in this bank?"
# user_input = "My income is around 2500 tomans.How do I calculate my customer score in this bank?"

# user_input = "من ۳ ساله که توی بانک حساب دارم. درآمد من معادل 5 میلیون تومانه. امتیاز من در بانک چقدر میشه؟"
user_input = "I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?"
# user_input = "How much money should I pay to pay all my loans to the bank?"

In [19]:
# Use the router chain to decide which formula chain to use.
router_result = router_chain({"question": user_input})
destination = router_result["destination"]
next_inputs = router_result["next_inputs"]

print("Router output:", router_result)
print("Selected destination:", destination)

# Check that the destination chain exists.
if destination not in destination_chains:
    raise ValueError(f"Destination '{destination}' not found in destination chains.")


# Use the selected formula chain to get the arithmetic expression.
chain_output = destination_chains[destination](next_inputs)
if isinstance(chain_output, dict) and "text" in chain_output:
    raw_formula_str = chain_output["text"]
else:
    raw_formula_str = chain_output

print("Raw formula output from chain:")
print(raw_formula_str)

# Call the extraction function here to clean the formula.
formula_str = extract_arithmetic_expression(raw_formula_str)
print("Extracted arithmetic expression:")
print(formula_str)


# Determine required parameters for the selected destination.
required_params = formula_required_params.get(destination, [])
print("Required parameters:", required_params)

# Extract any provided parameter values from the original question (or next_inputs["question"])
provided_params = extract_parameters_generic(next_inputs["question"], required_params)
print("Provided parameters:", provided_params)

# Identify any missing parameters and ask the user (here we use input() for demonstration).
missing_params = get_missing_parameters(required_params, provided_params)
if missing_params:
    for param in missing_params:
        value_str = input(f"Please enter a value for {param}: ")
        print(f"the value for {param} is {value_str}")
        try:
            provided_params[param] = float(value_str)
        except ValueError:
            raise ValueError(f"Invalid value provided for {param}.")

print("Final parameters for evaluation:", provided_params)


# Add math to the provided parameters so eval() knows what math.sqrt is.
provided_params['math'] = math

# Now, evaluate the extracted formula using eval() with the parameters.
try:
    result_value = eval(formula_str, provided_params)
except Exception as e:
    raise RuntimeError(f"Error evaluating the formula: {e}")

# Optionally, round the result to 2 decimal places.
if isinstance(result_value, float) and result_value != int(result_value):
    result_value = round(result_value, 2)
else:
    result_value = int(result_value)

print(f"Answer: {result_value}")


Router output: {'question': 'I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?', 'destination': 'score_calculation', 'next_inputs': {'question': 'I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?'}}
Selected destination: score_calculation


> Entering new LLMChain chain...
Prompt after formatting:
Customer Score Calculation:
For the question: I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?, please calculate the customer's score using the following formula:

    score = math.sqrt(amount_of_income * month_of_income) - ((years_in_bank)**3 / age)

Where:
  - amount_of_income: The total income amount of the customer.
  - month_of_income: The number of months corresponding to the income period.
  - years_in_bank: The number of years the customer has held an account with our bank.
  - age: The age of the customer.

Provide only the arithmetic expression (without additional expla

#### ********************************************

### Applying llm chain for extracting values of partameters

In [16]:
import math
import json
import re

def get_missing_parameters(required_params: list, provided: dict) -> list:
    return [p for p in required_params if p not in provided or provided[p] is None]

def extract_arithmetic_expression(text: str) -> str:
    r"""
    Extract a valid arithmetic expression from the LLM chain output.
    
    This function attempts to:
      1. Find a triple-backtick code block (optionally labeled "python"),
         remove any import lines, join remaining lines, and clean up escape sequences.
      2. If no code block is found, try to locate a LaTeX math block delimited by \[ and \].
      3. As a fallback, iterate over lines to find one that looks like an arithmetic expression.
    """
    # Try to extract a triple-backtick code block.
    code_block_match = re.search(r"```(?:python)?\s*(.*?)```", text, re.DOTALL)
    if code_block_match:
        code = code_block_match.group(1).strip()
        # Remove any lines starting with "import"
        lines = code.splitlines()
        filtered_lines = [line for line in lines if not line.strip().startswith("import")]
        code = " ".join(filtered_lines).strip()
        # Replace any backslash-underscore with underscore.
        code = code.replace(r"\_", "_")
        return code

    # If not found, try to extract a LaTeX math block.
    latex_match = re.search(r"\\\[(.*?)\\\]", text, re.DOTALL)
    if latex_match:
        expr = latex_match.group(1)
        expr = expr.replace("\n", " ").strip()
        expr = re.sub(r"\\text\{(.*?)\}", r"\1", expr)
        if "=" in expr:
            expr = expr.split("=", 1)[1].strip()
        expr = expr.replace(r"\times", "*")
        expr = expr.replace("{", "").replace("}", "")
        expr = re.sub(r"\\_", "_", expr)
        return expr

    # Fallback: iterate over lines, skip lines starting with import or code fences.
    for line in text.splitlines():
        line = line.strip()
        if line.startswith("import") or line.startswith("```"):
            continue
        line_clean = line.replace(r"\_", "_")
        if re.match(r"^[\d\w\.\+\-\*/\(\)\s]+$", line_clean):
            return line_clean
    return text

def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the JSON object from the LLM output.
    Finds the first '{' and the last '}' and parses that substring.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception as e:
        raise ValueError(f"Error parsing JSON: {e}")

def fill_missing_parameters(params: dict) -> dict:
    """
    For any parameter with a value of None, prompt the user to provide a value.
    """
    for param in params:
        if params[param] is None:
            value_str = input(f"Please enter a value for {param}: ")
            try:
                params[param] = float(value_str)
            except ValueError:
                raise ValueError(f"Invalid value provided for {param}.")
    return params

# ----------------------------------------------------------------------------
# Assume these components are defined elsewhere:
# - router_chain: chain to choose the formula chain.
# - destination_chains: dict mapping destination to formula chain.
# - formula_required_params: dict mapping destination to list of parameter names.
# - extraction_chains: dict mapping destination to dedicated parameter extraction chains.
# ----------------------------------------------------------------------------

# Example user input (for customer score calculation):
# user_input = "من 3 ساله که توی بانک حساب دارم. درآمد من معادل 5500 میلیون تومانه. امتیاز من در بانک چقدر میشه؟"
# user_input = " امتیاز من در بانک چقدر میشه؟"
user_input = "I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?"

# Step 1: Use the router chain to decide which formula chain to use.
router_result = router_chain({"question": user_input})
destination = router_result["destination"]
next_inputs = router_result["next_inputs"]

print("Router output:", router_result)
print("Selected destination:", destination)

if destination not in destination_chains:
    raise ValueError(f"Destination '{destination}' not found in destination chains.")

# Step 2: Retrieve and clean the formula.
chain_output = destination_chains[destination](next_inputs)
if isinstance(chain_output, dict) and "text" in chain_output:
    raw_formula_str = chain_output["text"]
else:
    raw_formula_str = chain_output

print("Raw formula output from chain:")
print(raw_formula_str)

formula_str = extract_arithmetic_expression(raw_formula_str)
print("Extracted arithmetic expression:")
print(formula_str)

# Step 3: Determine required parameters for the selected destination.
required_params = formula_required_params.get(destination, [])
print("Required parameters:", required_params)

# Use the dedicated extraction chain for this destination to extract parameters.
extraction_chains = {
    "interest_calculation": interest_extraction_chain,
    "score_calculation": score_extraction_chain,
    "loan_calculation": loan_extraction_chain
}

if destination in extraction_chains:
    extraction_result = extraction_chains[destination].run({"user_input": next_inputs["question"]})
    print("Parameter extraction result from LLM chain:")
    print(extraction_result)
    try:
        provided_params = parse_extraction_result(extraction_result)
    except Exception as e:
        raise ValueError(f"Error parsing extraction result: {e}")
else:
    provided_params = {}

print("Provided parameters (from LLM extraction):", provided_params)

# Ask the user for any missing parameters.
missing_params = get_missing_parameters(required_params, provided_params)
if missing_params:
    for param in missing_params:
        value_str = input(f"Please enter a value for {param}: ")
        print(f"the value for {param} is {value_str}")
        try:
            provided_params[param] = float(value_str)
        except ValueError:
            raise ValueError(f"Invalid value provided for {param}.")

print("Final parameters for evaluation:", provided_params)

# Step 4: Evaluate the formula.
provided_params['math'] = math  # Make math module available.

try:
    result_value = eval(formula_str, provided_params)
except Exception as e:
    raise RuntimeError(f"Error evaluating the formula: {e}")

if isinstance(result_value, float) and result_value != int(result_value):
    result_value = round(result_value, 2)
else:
    result_value = int(result_value)

print(f"Answer: {result_value}")


Router output: {'question': 'I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?', 'destination': 'score_calculation', 'next_inputs': {'question': 'I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?'}}
Selected destination: score_calculation


> Entering new LLMChain chain...
Prompt after formatting:
Customer Score Calculation:
For the question: I have account in the back for 3 years. My income is 45 M Tomans. How much is my score?, please calculate the customer's score using the following formula:

    score = (amount_of_income * month_of_income) - ((years_in_bank)**3 / age)

Where:
  - amount_of_income: The total income amount of the customer.
  - month_of_income: The number of months corresponding to the income period.
  - years_in_bank: The number of years the customer has held an account with our bank.
  - age: The age of the customer.

Provide only the arithmetic expression (without additional explanation) i

### Try to solve default value problem

#### We ask the user to accept the extracted value or replace it by the new one

In [ ]:
import math
import json
import re

def get_missing_parameters(required_params: list, provided: dict) -> list:
    # Return parameters that are missing (None) or not present at all.
    return [p for p in required_params if provided.get(p) is None]

def extract_arithmetic_expression(text: str) -> str:
    r"""
    Extract a valid arithmetic expression from the LLM chain output.
    
    This function attempts to:
      1. Find a triple-backtick code block (optionally labeled with any word),
         remove any import lines, and join remaining lines.
      2. Then, take only the first non-empty line of the code block as the expression.
      3. If not found, try to locate a LaTeX math block delimited by \[ and \].
      4. As a fallback, iterate over lines to find one that looks like an arithmetic expression.
    """
    # Try to extract a triple-backtick code block for any language.
    code_block_match = re.search(r"```(?:\w+)?\s*(.*?)```", text, re.DOTALL)
    if code_block_match:
        code = code_block_match.group(1).strip()
        # Remove lines starting with "import"
        lines = [line for line in code.splitlines() if not line.strip().startswith("import")]
        # Take only the first non-empty line.
        for line in lines:
            clean_line = line.strip()
            if clean_line:
                # Replace any backslash-underscore with underscore.
                clean_line = clean_line.replace(r"\_", "_")
                return clean_line
    # Try to extract a LaTeX math block.
    latex_match = re.search(r"\\\[(.*?)\\\]", text, re.DOTALL)
    if latex_match:
        expr = latex_match.group(1)
        expr = expr.replace("\n", " ").strip()
        expr = re.sub(r"\\text\{(.*?)\}", r"\1", expr)
        if "=" in expr:
            expr = expr.split("=", 1)[1].strip()
        expr = expr.replace(r"\times", "*")
        expr = expr.replace("{", "").replace("}", "")
        expr = re.sub(r"\\_", "_", expr)
        return expr
    # Fallback: iterate over lines.
    for line in text.splitlines():
        line = line.strip()
        if line.startswith("import") or line.startswith("```"):
            continue
        line_clean = line.replace(r"\_", "_")
        if re.match(r"^[\d\w\.\+\-\*/\(\)\s]+$", line_clean):
            return line_clean
    return text




# import json
import ast

def parse_extraction_result(result_str: str) -> dict:
    """
    Extract the dictionary from the LLM output.
    First, find the first '{' and the last '}'.
    Then try to parse that substring as JSON. 
    If that fails (e.g. due to single quotes), fall back to ast.literal_eval.
    """
    start = result_str.find('{')
    end = result_str.rfind('}') + 1
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in the output.")
    json_str = result_str[start:end]
    try:
        return json.loads(json_str)
    except Exception as e:
        # Fallback: try ast.literal_eval, which can handle Python dict syntax
        try:
            return ast.literal_eval(json_str)
        except Exception as e2:
            raise ValueError(f"Error parsing extraction result: {e2}")


def fill_or_override_parameters(provided_params: dict, required_params: list) -> dict:
    """
    For each required parameter, if a value exists, ask the user if they want to override it.
    If no value exists (i.e. None), prompt the user for a value.
    """
    for param in required_params:
        if provided_params.get(param) is None:
            # Parameter missing, ask user to input a value.
            value_str = input(f"Please enter a value for {param}: ")
            try:
                provided_params[param] = float(value_str)
            except ValueError:
                raise ValueError(f"Invalid value provided for {param}.")
        else:
            # Parameter exists; let the user override if desired.
            new_value_str = input(f"The extracted value for {param} is {provided_params[param]}. Press Enter to keep it, or enter a new value: ")
            if new_value_str.strip():
                try:
                    provided_params[param] = float(new_value_str)
                except ValueError:
                    raise ValueError(f"Invalid value provided for {param}.")
    return provided_params



In [38]:
# Example user input (for customer score calculation):
# user_input = "I have account in the bank for 3 years. My income is 45 M Tomans. How much is my score?"
# Or in Persian:
# user_input = "من 3 ساله که توی بانک حساب دارم. درآمد من معادل 5500 میلیون تومانه. امتیاز من در بانک چقدر میشه؟"

# user_input = "منمیخوام امتیازم رو توی بانک حساب کنم. سنم ۲۸ ساله و ۱۰ ماه پول تو حساب بانکیم گذاشتم"

# user_input = "من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره"

# user_input = "سود سپردم اگه ۱۸ ماه پولم تو حساب بانکیم باشه چقدر میشه؟"

user_input = "من ۲۵ میلیون از بانک ملت وام گرفتم. تا الان ۱۲ تا از اقساط رو به طور کامل پرداخت کردم. چقدر دیگه باید بدم تا تسویه بشه؟"

In [ ]:

# Step 1: Use the router chain.
router_result = router_chain({"question": user_input})
destination = router_result["destination"]
next_inputs = router_result["next_inputs"]

print("Router output:", router_result)
print("Selected destination:", destination)

if destination not in destination_chains:
    raise ValueError(f"Destination '{destination}' not found in destination chains.")

# Step 2: Retrieve and clean the formula.
chain_output = destination_chains[destination](next_inputs)
if isinstance(chain_output, dict) and "text" in chain_output:
    raw_formula_str = chain_output["text"]
else:
    raw_formula_str = chain_output

print("Raw formula output from chain:")
print(raw_formula_str)

formula_str = extract_arithmetic_expression(raw_formula_str)
print("Extracted arithmetic expression:")
print(formula_str)

#################################################################
## Fallback formula
import re

# Define fallback formulas for each destination.
fallback_formula_map = {
    "loan_calculation": "total_amount - (number_of_paid_loans * amount_per_loan)",
    "score_calculation": "math.sqrt(amount_of_income * month_of_income) - ((years_in_bank)**3 / age)",
    "interest_calculation": "((deposit_amount * months) / 12) * 0.25"
}

# Define a regex pattern that matches a clean arithmetic expression.
# This pattern allows letters (including underscores), digits, dot, plus, minus, asterisk, slash, parentheses, and whitespace.
allowed_pattern = r"^[\w\.\+\-\*/\(\)\s]+$"

# For all formulas, if the extracted formula is not clean, fall back to the default.
if destination in fallback_formula_map:
    if not re.match(allowed_pattern, formula_str):
        print("Extracted formula is not clean. Falling back to default expression.")
        formula_str = fallback_formula_map[destination]

print("Final cleaned arithmetic expression:")
print(formula_str)



####################################################################

# Step 3: Determine required parameters.
required_params = formula_required_params.get(destination, [])
print("Required parameters:", required_params)

# Use the dedicated extraction chain for parameters.
extraction_chains = {
    "interest_calculation": interest_extraction_chain,
    "score_calculation": score_extraction_chain,
    "loan_calculation": loan_extraction_chain
}

if destination in extraction_chains:
    extraction_result = extraction_chains[destination].run({"user_input": next_inputs["question"]})
    print("Parameter extraction result from LLM chain:")
    print(extraction_result)
    try:
        provided_params = parse_extraction_result(extraction_result)
    except Exception as e:
        raise ValueError(f"Error parsing extraction result: {e}")
else:
    provided_params = {}

print("Provided parameters (from LLM extraction):", provided_params)

# Step 4: Ask the user to fill or override parameters.
provided_params = fill_or_override_parameters(provided_params, required_params)
print("Final parameters for evaluation:", provided_params)

# Step 5: Evaluate the formula.
provided_params['math'] = math  # Make math module available.
try:
    result_value = eval(formula_str, provided_params)
except Exception as e:
    raise RuntimeError(f"Error evaluating the formula: {e}")

if isinstance(result_value, float) and result_value != int(result_value):
    result_value = round(result_value, 2)
else:
    result_value = int(result_value)

print(f"Answer: {result_value}")



Router output: {'question': 'من ۲۵ میلیون از بانک ملت وام گرفتم. تا الان ۱۲ تا از اقساط رو به طور کامل پرداخت کردم. چقدر دیگه باید بدم تا تسویه بشه؟', 'destination': 'loan_calculation', 'next_inputs': {'question': 'من ۲۵ میلیون از بانک ملت وام گرفتم. تا الان ۱۲ تا از اقساط رو به طور کامل پرداخت کردم. چقدر دیگه باید بدم تا تسویه بشه؟'}}
Selected destination: loan_calculation


> Entering new LLMChain chain...
Prompt after formatting:
Loan Calculation Expert:
For the question: من ۲۵ میلیون از بانک ملت وام گرفتم. تا الان ۱۲ تا از اقساط رو به طور کامل پرداخت کردم. چقدر دیگه باید بدم تا تسویه بشه؟, please calculate the amount of money required to pay off all loans using the following formula:

    amount_to_pay = total_amount - (number_of_paid_loans * amount_per_loan)

Where:
  - total_amount: The total amount of money owed on all loans.
  - number_of_paid_loans: The number of loans that have already been paid off.
  - amount_per_loan: The fixed payment amount for each paid loan.

Provide o